<a href="https://colab.research.google.com/github/RK22000/FOLd/blob/main/FOLd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This document is divided into 3 sections:

1. Helper Methods

        Has all the helper methods and their minor tests. (Skipable)
1. Main Code
    
        Has the code for the main brains of the First Order Logic.
1. Tests
    
        Shows the FOL engine at work with test cases. Can be checked before reading other sections.



# Helper methods

In [ ]:
import re

In [ ]:
re.findall(r'\(.*\)', "Smart(x)(y)")

['(x)(y)']

In [ ]:
re.findall(r'\([^(]*\)', "Smart(x)(y)")

['(x)', '(y)']

In [ ]:
#match = re.findall(r'\((?P<arguments>[^()]*)\)', "Smart(x, y) Clever(p, (q))")
b1 = r'\(([^)(]*)\)'
b2 = r'\(([^)(]*' + r'\([^)(]\)' + r'[^()]*)\)'

match = re.findall(b1+'|'+b2, "Smart(x, y) Clever(p, (q))")
match#.group("arguments")

[('x, y', ''), ('', 'p, (q)')]

In [ ]:
import re
def getFuncArgs(predicate: str):
    """Get the first function and its set of arguments in the predicate"""
    regex = re.compile(r'(?P<negation>~?)(?P<function>\w+)\((?P<arguments>[^()]*)\)')
    match = regex.search(predicate)
    arguments = match.group("arguments").split(',')
    for i, argument in enumerate(arguments):
        arguments[i] = argument.strip()
    func = match.group("function")
    negation = match.group("negation")
    return func, arguments, negation


In [ ]:
getFuncArgs("~Smart(x, y)")

('Smart', ['x', 'y'], '~')

In [ ]:
import re
def isFuncArgs(predicate: str):
    """Get the first function and its set of arguments in the predicate"""
    regex = re.compile(r'^(?P<function>~?\w+)\((?P<arguments>[^()]*)\)$')
    match = regex.search(predicate)
    return True if match else False


In [ ]:
isFuncArgs("Smart(x)^Clever(x)")

False

In [ ]:
isFuncArgs("abc")

False

In [ ]:
def validTerm(term: str):
    regex = re.compile(r"^[A-Za-z]+$")
    return True if regex.search(term) else False

def groundTerm(term: str):
    if not validTerm(term=term): return False
    regex = re.compile(r"^[a-z]+$")
    return True if regex.search(term) else False

def constantTerm(term: str):
    if not validTerm(term=term): return False
    regex = re.compile(r"^[A-Z][A-Za-z]+$")
    return True if regex.search(term) else False

In [ ]:
groundTerm('x'), constantTerm('RahulKan'), validTerm('XyZ')

(True, True, True)

In [ ]:
def swapable(f_terms: list, p_terms: list):
    count=0
    if not len(f_terms) == len(p_terms): return False
    for f_term, p_term in zip(f_terms, p_terms):
        if groundTerm(f_term) or f_term==p_term:
            count+=1
    return count==len(f_terms)

In [ ]:
None == True, None == False

(False, False)

In [ ]:
def isRule(sentence: str):
    return sentence.count("=>") == 1

In [ ]:
isRule(" a => b")

True

In [ ]:
def getCondImpl(sentence: str):
    condition, implication = sentence.split("=>")
    return condition.strip(), implication.strip()

In [ ]:
getCondImpl(" a ^ x=> b ")

('a ^ x', 'b')

In [ ]:
def swap_list(f_terms: list, p_terms: list):
    swap = dict()
    if not len(f_terms) == len(p_terms): return swap
    for f_term, p_term in zip(f_terms, p_terms):
        if groundTerm(f_term):
            swap[f_term] = p_term
    return swap

In [ ]:
def replace_var(var: str, rpl: str, sentence:str) -> str:
    def swap(match: re.Match):
        #print(f'Match {match}')
        return match.group()[0] + rpl + match.group()[-1]
    wrapper = r"[\s)(,]"
    pattern = wrapper + f"(?P<var>{var})" + wrapper
    return re.sub(pattern=pattern, repl=swap, string=sentence)

In [ ]:
replace_var("y", "Azula", "Prodigy(y)")

Match <re.Match object; span=(7, 10), match='(y)'>


'Prodigy(Azula)'

In [ ]:
def conjunction(sentence: str) -> bool:
    conjuncts = set( [conj.strip() for conj in sentence.split("^")] )
    return "^" in sentence and '' not in conjuncts

In [ ]:
conjunction(" a ^ B  ^ ^ x")

False

In [ ]:
def getConjuncts(sentence: str) -> set:
    return set([conj.strip() for conj in sentence.split("^")])


In [ ]:
getConjuncts(" a ^ B ^ x ")

{'B', 'a', 'x'}

In [ ]:
def unifyable(f_terms: list, p_terms: list):
    count=0
    if not len(f_terms) == len(p_terms): return False
    for f_term, p_term in zip(f_terms, p_terms):
        if groundTerm(f_term) or groundTerm(p_term) or f_term==p_term:
            count+=1
    return count==len(f_terms)

In [ ]:
unifyable(["Aang"], ["x"]), unifyable(["g"], ["Aang"])

(True, True)

In [ ]:
def unified_arguments(f_terms: list, p_terms: list):
    swap = dict()
    if not len(f_terms) == len(p_terms): return swap
    for f_term, p_term in zip(f_terms, p_terms):
        if groundTerm(f_term):
            swap[f_term] = p_term
        elif groundTerm(p_term):
            swap[p_term] = f_term
    return swap

# Main Code

In [ ]:
class Fol_engine:
    def __init__(self, facts: set = set(), rules: set = set()) -> None:
        self.facts = facts
        self.rules = rules

    def trueCase(self, predicate: str):

        if conjunction(sentence=predicate):
            conjuncts = getConjuncts(sentence=predicate)
            print(f"Conjunct of {conjuncts}")
            result = True
            for conjunct in conjuncts:
                val = self.trueCase(conjunct)
                if val == False: # val may be False or None
                    return False
                elif val is None:
                    result = None
            return result

        else:
            cases = self.getCases(predicate=predicate)
            if predicate in cases[True]: return True
            elif predicate in cases[False]: return False
            else: return None
        


    def getCases(self, predicate: str):

        if isFuncArgs(predicate):
            p_func, p_arguments, p_negation = getFuncArgs(predicate)
            print(f"Getting cases for {predicate}")
            true_cases = set()
            false_cases = set()
            cases = {True: true_cases, False: false_cases}

            for fact in self.facts:
                if isFuncArgs(fact):
                    f_func, f_arguments, f_negation = getFuncArgs(fact)
                    if f_func==p_func and len(p_arguments)==len(f_arguments):
                        print(f'Fact Function match \'{fact}\'')
                        if unifyable(f_terms=f_arguments, p_terms=p_arguments):
                            print(f'Unifyable with {predicate}')
                            swap = unified_arguments(f_terms=f_arguments, p_terms=p_arguments)
                            unified = predicate
                            for var in swap:
                                unified = replace_var(var=var, rpl=swap[var], sentence=unified)
                            mod = "" if f_negation==p_negation else "not "
                            print(f'After unification {mod}{unified}')
                            cases[f_negation==p_negation].add(unified)
                        else:
                            print(f'Not unifyable with {predicate}')

            for rule in self.rules:
                if isRule(rule):
                    condition, implication = getCondImpl(rule)
                    if isFuncArgs(implication):
                        i_func, i_arguments, i_negation = getFuncArgs(implication)
                        if i_func==p_func and len(i_arguments)==len(p_arguments):
                            print(f'Rule match \'{rule}\' for \'{predicate}\'')
                            
                            if unifyable(f_terms=i_arguments, p_terms=p_arguments):
                                print(f'{implication} unifyable with {predicate}')
                                swap = unified_arguments(f_terms=i_arguments, p_terms=p_arguments)
                                unified = predicate
                                for var in swap:
                                    unified = replace_var(var=var, rpl=swap[var], sentence=unified)
                                    rule = replace_var(var=var, rpl=swap[var], sentence=rule)
                                print(f'After unification {rule}')
                                condition, implication = getCondImpl(rule)
                                sub_cases = self.getCases(condition)
                                if condition in sub_cases[True]:
                                    _, _, i_negation = getFuncArgs(implication)
                                    cases[i_negation==p_negation].add(unified)
                                print(sub_cases)

            return cases





    def isTrue(self, predicate: str):

        if conjunction(sentence=predicate):
            conjuncts = getConjuncts(sentence=predicate)
            print(f"Conjunct of {conjuncts}")
            result = True
            for conjunct in conjuncts:
                val = self.isTrue(conjunct)
                if val == False: # val may be False or None
                    return False
                elif val is None:
                    result = None
            return result
        
        elif isFuncArgs(predicate):
            p_func, p_arguments, p_negation = getFuncArgs(predicate)

            for arg in p_arguments:
                if groundTerm(arg):
                    print(f'Predicate {predicate} has groundterm {arg}')
                    #return

            print(f"Checking {predicate}")

            for fact in self.facts:
                if isFuncArgs(fact):
                    f_func, f_arguments, f_negation = getFuncArgs(fact)
                    if f_func==p_func and len(p_arguments)==len(f_arguments):
                        print(f'Fact Function match \'{p_func}\'')
                        print(f'p_arguments {p_arguments}, f_arguments {f_arguments}')

                        if swapable(f_terms=f_arguments, p_terms=p_arguments):
                            mod = "" if p_negation == f_negation else "not "
                            print(f'Fact \'{fact}\' |=> {mod}{predicate}')
                            return p_negation == f_negation
                        else:
                            print(f'Fact \'{fact}\' ?=> {predicate}')

            for rule in self.rules:
                if isRule(rule):
                    condition, implication = getCondImpl(rule)
                    if isFuncArgs(implication):
                        i_func, i_arguments, i_negation = getFuncArgs(implication)
                        if i_func==p_func and len(i_arguments)==len(p_arguments):
                            print(f'Rule match \'{rule}\' for \'{predicate}\'')

                            if swapable(f_terms=i_arguments, p_terms=p_arguments):
                                swap = swap_list(f_terms=i_arguments, p_terms=p_arguments)
                                print(f'Swaping {swap}')

                                for var in swap:
                                    condition = replace_var(var=var, rpl=swap[var], sentence=condition)
                                    rule = replace_var(var=var, rpl=swap[var], sentence=rule)
                                    #condition = re.sub(var, swap[var], condition)
                                    #rule = re.sub(var, swap[var], rule)
                                
                                print(f'Swapped condition \'{condition}\'')
                                result = self.isTrue(condition)
                                if result == True:
                                    print(f'Rule {rule}')
                                    return result if p_negation == i_negation else not result
        return None

# Tests

## Test 8

These tests are not cleared. They can be ignored. If you are simplly checking what has been done.

If predicate has ground terms, then is doesn't make sense to ask is_true(predicate)

Instead ask for a list of true predicates with ground terms filled in.

In [ ]:
print("Test 8.2.1")
def test():
    facts = {
        "TeamAvatarMember(Sokka)",
        "Scarcastic(Sokka)",
        "Prodigy(Azula)",
        "Villan(Azula)",
        "Lethal(Azula)",
    }
    rules = {
        "TeamAvatarMember(x) ^ Scarcastic(x) => Funny(x)",
        "Villan(y) ^ Lethal(y) ^ Prodigy(y) => Scary(y)", # Prodigy(y) swap y will swap y in both name and argument
    }
    engine_1 = Fol_engine(facts=facts, rules=rules)
    print(engine_1.trueCase("Funny(Sokka) ^ Scary(Azula)"))

test()

In [ ]:
print("Test 8.0.3")

print("\n\nTest 1")
engine_1 = Fol_engine(facts={"Smart(Sokka)", "Smart(Katara)", "Funny(Sokka)", "~Funny(Katara)"})
print( engine_1.trueCase("Funny(Sokka)") )

print("\n\nTest 2")
print( engine_1.trueCase("~Funny(Sokka)") )

print("\n\nTest 3")
print( engine_1.trueCase("Funny(Katara)") )

print("\n\nTest 4")
print( engine_1.trueCase("Funny(Sokka) ^ ~Funny(Katara)") )

Test 8.0.3


Test 1
Getting cases for Funny(Sokka)
Fact Function match '~Funny(Katara)'
Not unifyable with Funny(Sokka)
Fact Function match 'Funny(Sokka)'
Unifyable with Funny(Sokka)
After unification Funny(Sokka)
True


Test 2
Getting cases for ~Funny(Sokka)
Fact Function match '~Funny(Katara)'
Not unifyable with ~Funny(Sokka)
Fact Function match 'Funny(Sokka)'
Unifyable with ~Funny(Sokka)
After unification not ~Funny(Sokka)
False


Test 3
Getting cases for Funny(Katara)
Fact Function match '~Funny(Katara)'
Unifyable with Funny(Katara)
After unification not Funny(Katara)
Fact Function match 'Funny(Sokka)'
Not unifyable with Funny(Katara)
False


Test 4
Conjunct of {'~Funny(Katara)', 'Funny(Sokka)'}
Getting cases for ~Funny(Katara)
Fact Function match '~Funny(Katara)'
Unifyable with ~Funny(Katara)
After unification ~Funny(Katara)
Fact Function match 'Funny(Sokka)'
Not unifyable with ~Funny(Katara)
Getting cases for Funny(Sokka)
Fact Function match '~Funny(Katara)'
Not unifyable with Fu

In [ ]:
print("Test 8.0.2")

print("\n\nTest 1")
engine_1 = Fol_engine(facts={"Smart(Sokka)", "Smart(Katara)", "Funny(Sokka)"})
print( engine_1.getCases("Funny(x)") )

print("\n\nTest 2")
print( engine_1.getCases("Smart(x)") )

print("\n\nTest 3")
print( engine_1.getCases("Smart(x) ^ Funny(x)") )

Test 8.0.2


Test 1
Getting cases for Funny(x)
Fact Function match 'Funny(Sokka)'
Unifyable with Funny(x)
After unification Funny(Sokka)
{True: {'Funny(Sokka)'}, False: set()}


Test 2
Getting cases for Smart(x)
Fact Function match 'Smart(Katara)'
Unifyable with Smart(x)
After unification Smart(Katara)
Fact Function match 'Smart(Sokka)'
Unifyable with Smart(x)
After unification Smart(Sokka)
{True: {'Smart(Katara)', 'Smart(Sokka)'}, False: set()}


Test 3
None


In [ ]:
print("Test 8.0.1")

print("Test 8.0.1.A")
engine_1 = Fol_engine(facts={"Hostile(FireNation)"}, rules={"Hostile(x) => Enemy(x)"})
print( engine_1.getCases("Enemy(FireNation)") )

print("\n\nTest 8.0.1.B")
engine_1 = Fol_engine(facts={"Hostile(FireNation)"}, rules={"Hostile(x) => Enemy(x)"})
print( engine_1.getCases("~Enemy(FireNation)") )

print("\n\nTest 8.0.1.C")
engine_1 = Fol_engine(facts={"Friendly(AirNomads)"}, rules={"Friendly(x) => ~Enemy(x)"})
print( engine_1.getCases("Enemy(AirNomads)") )

Test 8.0.1
Test 8.0.1.A
Getting cases for Enemy(FireNation)
Rule match 'Hostile(x) => Enemy(x)' for 'Enemy(FireNation)'
Enemy(x) unifyable with Enemy(FireNation)
After unification Hostile(FireNation) => Enemy(FireNation)
Getting cases for Hostile(FireNation)
Fact Function match 'Hostile(FireNation)'
Unifyable with Hostile(FireNation)
After unification Hostile(FireNation)
{True: {'Hostile(FireNation)'}, False: set()}
{True: {'Enemy(FireNation)'}, False: set()}


Test 8.0.1.B
Getting cases for ~Enemy(FireNation)
Rule match 'Hostile(x) => Enemy(x)' for '~Enemy(FireNation)'
Enemy(x) unifyable with ~Enemy(FireNation)
After unification Hostile(FireNation) => Enemy(FireNation)
Getting cases for Hostile(FireNation)
Fact Function match 'Hostile(FireNation)'
Unifyable with Hostile(FireNation)
After unification Hostile(FireNation)
{True: {'Hostile(FireNation)'}, False: set()}
{True: set(), False: {'~Enemy(FireNation)'}}


Test 8.0.1.C
Getting cases for Enemy(AirNomads)
Rule match 'Friendly(x) => 

In [ ]:
print("Test 8.0")

facts={
    "TeamAvatarMember(Aang)", 
    "TeamAvatarMember(Sokka)", 
    "TeamAvatarMember(Katara)", 
    "TeamAvatarMember(Toph)", 
    "TeamAvatarMember(Suki)",
    "~TeamAvatarMember(Bumi)",
    "~TeamAvatarMember(Iroh)"
}

engine_1 = Fol_engine(facts=facts)
engine_1.isTrue("TeamAvatarMember(x)")
print( engine_1.getCases("TeamAvatarMember(x)") )

print("\n\nTest 8.0.B")
print( engine_1.getCases("~TeamAvatarMember(x)") )

print("\n\nTest 8.0.C")
print( engine_1.getCases("TeamAvatarMember(Toph)") )

print("\n\nTest 8.0.D")
print( engine_1.getCases("TeamAvatarMember(Bumi)") )

print("\n\nTest 8.0.E")
print( engine_1.getCases("~TeamAvatarMember(Iroh)") )


Test 8.0
Predicate TeamAvatarMember(x) has groundterm x
Checking TeamAvatarMember(x)
Fact Function match 'TeamAvatarMember'
p_arguments ['x'], f_arguments ['Sokka']
Fact 'TeamAvatarMember(Sokka)' ?=> TeamAvatarMember(x)
Fact Function match 'TeamAvatarMember'
p_arguments ['x'], f_arguments ['Bumi']
Fact '~TeamAvatarMember(Bumi)' ?=> TeamAvatarMember(x)
Fact Function match 'TeamAvatarMember'
p_arguments ['x'], f_arguments ['Suki']
Fact 'TeamAvatarMember(Suki)' ?=> TeamAvatarMember(x)
Fact Function match 'TeamAvatarMember'
p_arguments ['x'], f_arguments ['Aang']
Fact 'TeamAvatarMember(Aang)' ?=> TeamAvatarMember(x)
Fact Function match 'TeamAvatarMember'
p_arguments ['x'], f_arguments ['Iroh']
Fact '~TeamAvatarMember(Iroh)' ?=> TeamAvatarMember(x)
Fact Function match 'TeamAvatarMember'
p_arguments ['x'], f_arguments ['Toph']
Fact 'TeamAvatarMember(Toph)' ?=> TeamAvatarMember(x)
Fact Function match 'TeamAvatarMember'
p_arguments ['x'], f_arguments ['Katara']
Fact 'TeamAvatarMember(Katara)' 

In [ ]:
print("Test 8.1")
def test():
    facts = {
        "American(CoWest)",
        "Sells(CoWest, Missile, Nono)",
        "Hostile(Nono)",
        "Has(Nono, Missile)",
        "Weapon(Missile)",

    }
    rules = {
        "American(x) ^ Weapon(y) ^ Enemy(z) ^ Sold(x, y, z) => Criminal(x)",
        "Hostile(w) => Enemy(w)",
        "Sells(p, q, r) ^ Has(r, q) => Sold(p, q, r)"
    }
    engine_1 = Fol_engine(facts=facts, rules=rules)
    print(engine_1.isTrue("Criminal(CoWest)"))

test()

Test 8.1
Checking Criminal(CoWest)
Rule match 'American(x) ^ Weapon(y) ^ Enemy(z) ^ Sold(x, y, z) => Criminal(x)' for 'Criminal(CoWest)'
Swaping {'x': 'CoWest'}
Swapped condition 'American(CoWest) ^ Weapon(y) ^ Enemy(z) ^ Sold(CoWest, y, z)'
Conjunct of {'Enemy(z)', 'Weapon(y)', 'Sold(CoWest, y, z)', 'American(CoWest)'}
Predicate Enemy(z) has groundterm z
Checking Enemy(z)
Rule match 'Hostile(w) => Enemy(w)' for 'Enemy(z)'
Swaping {'w': 'z'}
Swapped condition 'Hostile(z)'
Predicate Hostile(z) has groundterm z
Checking Hostile(z)
Fact Function match 'Hostile'
p_arguments ['z'], f_arguments ['Nono']
Fact 'Hostile(Nono)' ?=> Hostile(z)
Predicate Weapon(y) has groundterm y
Checking Weapon(y)
Fact Function match 'Weapon'
p_arguments ['y'], f_arguments ['Missile']
Fact 'Weapon(Missile)' ?=> Weapon(y)
Predicate Sold(CoWest, y, z) has groundterm y
Predicate Sold(CoWest, y, z) has groundterm z
Checking Sold(CoWest, y, z)
Rule match 'Sells(p, q, r) ^ Has(r, q) => Sold(p, q, r)' for 'Sold(CoWest,

## Test 7

These tests have been passed and they demonstrate the current capabalities of my current FOL engine.

In [ ]:
print("Test 7.1")
def test():
    facts = {
        "TeamAvatarMember(Sokka)",
        "Scarcastic(Sokka)",
        "Prodigy(Azula)",
        "Villan(Azula)",
        "Lethal(Azula)",
    }
    rules = {
        "TeamAvatarMember(x) ^ Scarcastic(x) => Funny(x)",
        "Villan(y) ^ Lethal(y) ^ Prodigy(y) => Scary(y)", # Prodigy(y) swap y will swap y in both name and argument
    }
    engine_1 = Fol_engine(facts=facts, rules=rules)
    print(engine_1.isTrue("Funny(Sokka) ^ Scary(Azula)"))

test()

Test 7.1
Conjunct of {'Scary(Azula)', 'Funny(Sokka)'}
Checking Scary(Azula)
Rule match 'Villan(y) ^ Lethal(y) ^ Prodigy(y) => Scary(y)' for 'Scary(Azula)'
Swaping {'y': 'Azula'}
Swapped condition 'Villan(Azula) ^ Lethal(Azula) ^ Prodigy(Azula)'
Conjunct of {'Villan(Azula)', 'Lethal(Azula)', 'Prodigy(Azula)'}
Checking Villan(Azula)
Fact Function match 'Villan'
p_arguments ['Azula'], f_arguments ['Azula']
Fact 'Villan(Azula)' |=> Villan(Azula)
Checking Lethal(Azula)
Fact Function match 'Lethal'
p_arguments ['Azula'], f_arguments ['Azula']
Fact 'Lethal(Azula)' |=> Lethal(Azula)
Checking Prodigy(Azula)
Fact Function match 'Prodigy'
p_arguments ['Azula'], f_arguments ['Azula']
Fact 'Prodigy(Azula)' |=> Prodigy(Azula)
Rule Villan(Azula) ^ Lethal(Azula) ^ Prodigy(Azula) => Scary(Azula)
Checking Funny(Sokka)
Rule match 'TeamAvatarMember(x) ^ Scarcastic(x) => Funny(x)' for 'Funny(Sokka)'
Swaping {'x': 'Sokka'}
Swapped condition 'TeamAvatarMember(Sokka) ^ Scarcastic(Sokka)'
Conjunct of {'TeamAva

## Tests 6 and below

In [ ]:
# Test 6
print("\nTest 1")
engine_1 = Fol_engine(facts={"Smart(Sokka)", "Funny(Sokka)"})
engine_1.isTrue("Smart(Sokka) ^ Funny(Sokka)")

print("\nTest 2")
engine_1 = Fol_engine(facts={"Smart(x)", "Funny(Sokka)"})
engine_1.isTrue("Smart(Sokka) ^ Funny(Sokka)")

print("\nTest 3")
engine_1 = Fol_engine(facts={"Smart(x)", "Funny(Sokka)"})
engine_1.isTrue("~Smart(Sokka) ^ Funny(Sokka)")

print("\nTest 4")
engine_1 = Fol_engine(facts={"Smart(x)", "~Funny(Sokka)"})
engine_1.isTrue("Smart(Sokka) ^ Funny(Sokka)")

print("\nTest 5")
engine_1 = Fol_engine(facts={"Smart(x)", "Funny(Sokka)"})
print(engine_1.isTrue("Smart(Sokka) ^ Funny(Azula)"))

print("\nTest 6")
engine_1 = Fol_engine(facts={"Smart(x)", "Funny(Sokka)", "~Funny(Azula)"})
print(engine_1.isTrue("Smart(Sokka) ^ Funny(Azula)"))


Test 1
Conjunct of {'Smart(Sokka)', 'Funny(Sokka)'}
Checking Smart(Sokka)
Fact Function match 'Smart'
p_arguments ['Sokka'], f_arguments ['Sokka']
Fact 'Smart(Sokka)' |=> Smart(Sokka)
Checking Funny(Sokka)
Fact Function match 'Funny'
p_arguments ['Sokka'], f_arguments ['Sokka']
Fact 'Funny(Sokka)' |=> Funny(Sokka)

Test 2
Conjunct of {'Smart(Sokka)', 'Funny(Sokka)'}
Checking Smart(Sokka)
Fact Function match 'Smart'
p_arguments ['Sokka'], f_arguments ['x']
Fact 'Smart(x)' |=> Smart(Sokka)
Checking Funny(Sokka)
Fact Function match 'Funny'
p_arguments ['Sokka'], f_arguments ['Sokka']
Fact 'Funny(Sokka)' |=> Funny(Sokka)

Test 3
Conjunct of {'~Smart(Sokka)', 'Funny(Sokka)'}
Checking ~Smart(Sokka)
Fact Function match 'Smart'
p_arguments ['Sokka'], f_arguments ['x']
Fact 'Smart(x)' |=> not ~Smart(Sokka)

Test 4
Conjunct of {'Smart(Sokka)', 'Funny(Sokka)'}
Checking Smart(Sokka)
Fact Function match 'Smart'
p_arguments ['Sokka'], f_arguments ['x']
Fact 'Smart(x)' |=> Smart(Sokka)
Checking Funn

In [ ]:
# Test 5

print("\nTest 1")
engine_1 = Fol_engine(facts={"Clever(Rahul)"}, rules={"Clever(x) => Smart(x)"})
print( engine_1.isTrue("Smart(Rahul)") )


print("\nTest 2")
engine_1 = Fol_engine(facts={"Clever(y)"}, rules={"Clever(x) => Smart(x)"})
print( engine_1.isTrue("Smart(Rahul)") )

print("\nTest 3")
engine_1 = Fol_engine(facts={"Clever(y)"}, rules={"Clever(x) => ~Silly(x)"})
print( engine_1.isTrue("Silly(Rahul)") )

print("\nTest 4")
engine_1 = Fol_engine(facts={"~Clever(Rahul)"}, rules={"Clever(x) => Smart(x)"})
print( engine_1.isTrue("Smart(Rahul)") )

print("\nTest 5")
engine_1 = Fol_engine(facts={"Clever(Rahul)"}, rules={"~Clever(x) => Smart(x)"})
print( engine_1.isTrue("Smart(Rahul)") )


Test 1
Rule match 'Clever(x) => Smart(x)' for 'Smart(Rahul)'
Swaping {'x': 'Rahul'}
Swapped condition 'Clever(Rahul)'
Fact Function match 'Clever'
p_arguments ['Rahul'], f_arguments ['Rahul']
Fact 'Clever(Rahul)' |=> Clever(Rahul)
Rule Clever(Rahul) => Smart(Rahul)
True

Test 2
Rule match 'Clever(x) => Smart(x)' for 'Smart(Rahul)'
Swaping {'x': 'Rahul'}
Swapped condition 'Clever(Rahul)'
Fact Function match 'Clever'
p_arguments ['Rahul'], f_arguments ['y']
Fact 'Clever(y)' |=> Clever(Rahul)
Rule Clever(Rahul) => Smart(Rahul)
True

Test 3
Rule match 'Clever(x) => ~Silly(x)' for 'Silly(Rahul)'
Swaping {'x': 'Rahul'}
Swapped condition 'Clever(Rahul)'
Fact Function match 'Clever'
p_arguments ['Rahul'], f_arguments ['y']
Fact 'Clever(y)' |=> Clever(Rahul)
Rule Clever(Rahul) => ~Silly(Rahul)
False

Test 4
Rule match 'Clever(x) => Smart(x)' for 'Smart(Rahul)'
Swaping {'x': 'Rahul'}
Swapped condition 'Clever(Rahul)'
Fact Function match 'Clever'
p_arguments ['Rahul'], f_arguments ['Rahul']
Fact

In [ ]:
# Test 4

print("\nTest 1")
engine_1 = Fol_engine(facts={"~Clever(Rahul)"})
print( engine_1.isTrue("Clever(Rahul)") )

print("\nTest 2")
engine_1 = Fol_engine(facts={"~Clever(Rahul)"})
print( engine_1.isTrue("~Clever(Rahul)") )

print("\nTest 3")
engine_1 = Fol_engine(facts={"~Clever(x)"})
print( engine_1.isTrue("Clever(Rahul)") )

print("\nTest 4")
engine_1 = Fol_engine(facts={"~Clever(x)"})
print( engine_1.isTrue("~Clever(Rahul)") )

print("\nTest 5")
engine_1 = Fol_engine(facts={"~Clever(Rahul)"})
print( engine_1.isTrue("Clever(x)") )

print("\nTest 6")
engine_1 = Fol_engine(facts={"~Clever(Rahul)"})
print( engine_1.isTrue("~Clever(x)") )


Test 1
Fact Function match 'Clever'
p_arguments ['Rahul'], f_arguments ['Rahul']
Fact '~Clever(Rahul)' |=> not Clever(Rahul)
False

Test 2
Fact Function match 'Clever'
p_arguments ['Rahul'], f_arguments ['Rahul']
Fact '~Clever(Rahul)' |=> ~Clever(Rahul)
True

Test 3
Fact Function match 'Clever'
p_arguments ['Rahul'], f_arguments ['x']
Fact '~Clever(x)' |=> not Clever(Rahul)
False

Test 4
Fact Function match 'Clever'
p_arguments ['Rahul'], f_arguments ['x']
Fact '~Clever(x)' |=> ~Clever(Rahul)
True

Test 5
Fact Function match 'Clever'
p_arguments ['x'], f_arguments ['Rahul']
None

Test 6
Fact Function match 'Clever'
p_arguments ['x'], f_arguments ['Rahul']
None


In [ ]:
 # Test 3
'''

'''

print('Test 1')
engine_1 = Fol_engine(facts={"Has(x, Pen)"})
engine_1.isTrue("Has(Rahul, Pen)")

print('Test 2')
engine_1 = Fol_engine(facts={"OwnedBy(Pen, x)"})
engine_1.isTrue("OwnedBy(Pen, Rahul)")

Test 1
Fact Function match 'Has'
p_arguments ['Rahul', 'Pen'], f_arguments ['x', 'Pen']
Fact 'Has(x, Pen)' |=> Has(Rahul, Pen)
Test 2
Fact Function match 'OwnedBy'
p_arguments ['Pen', 'Rahul'], f_arguments ['Pen', 'x']
Fact 'OwnedBy(Pen, x)' |=> OwnedBy(Pen, Rahul)


True

In [ ]:
 # Test 2
'''
Engine should unify "Smart(Rahul)" with "Smart(x)"
first recognize "Smart(x)" is FuncArg
then recognize argument x is ground term, so itmight be unified with something
try to unify:
    success: is this the result?
    fail: move on
'''

print('Test 1')
engine_1 = Fol_engine(facts={"Smart(x)"})  # for all x, Smart(x) is true
engine_1.isTrue("Smart(Rahul)")

print("\nTest 2")
engine_1 = Fol_engine(facts={"Clever(x)", "Smart(y)"})
engine_1.isTrue("Smart(Rahul)")

print("\nTest 3")
engine_1 = Fol_engine(facts={"Clever(x)"})
engine_1.isTrue("Smart(Rahul)") == None

Test 1
Fact Function match 'Smart'
p_arguments ['Rahul'], f_arguments ['x']
Fact 'Smart(x)' |=> Smart(Rahul)

Test 2
Fact Function match 'Smart'
p_arguments ['Rahul'], f_arguments ['y']
Fact 'Smart(y)' |=> Smart(Rahul)

Test 3


True

In [ ]:
# Test 1
engine_1 = Fol_engine(facts={"Smart(Rahul)"})
engine_1.isTrue("Smart(Rahul)")

Fact Function match 'Smart'
p_arguments ['Rahul'], f_arguments ['Rahul']
Fact 'Smart(Rahul)' |=> Smart(Rahul)


True